In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
import pandas as pd
from sklearn.metrics import roc_auc_score

In [2]:
def calculate_ece(confidences, accuracies, n_bins=10) -> float:
    """
    Calculate the expected calibration error (ECE) given a list of confidence scores (0-1) and accuracy scores (0 or 1).
    """
    df = pd.DataFrame({"conf": confidences, "acc": accuracies}).dropna()

    confidences = torch.tensor(df["conf"].tolist())
    accuracies = torch.tensor(df["acc"].tolist())
    bin_boundaries = torch.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

    ece = torch.zeros(1)
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        # Calculated |confidence - accuracy| in each bin
        in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
        prop_in_bin = in_bin.float().mean()
        if prop_in_bin.item() > 0:
            accuracy_in_bin = accuracies[in_bin].float().mean()
            avg_confidence_in_bin = confidences[in_bin].mean()
            ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
    
    return ece.item()

# Drop NA on an Individual Basis

In [3]:
stats_df = pd.DataFrame()
for file in os.listdir("../results"):
    if file.endswith("csv"):
        df = pd.read_csv("../results/" + file)
        acc_no_na = df[["extracted_answer", "correct_answer"]].dropna()
        new_row = pd.DataFrame({
            "Model": [file.replace("mmlu_pro_", "").replace("_eval_all_None.csv", "").strip().capitalize()],
            "Missing answer": [df["extracted_answer"].isna().sum()],
            "Missing verbal numerical confidence": [df["verbal_numerical_confidence"].isna().sum()],
            "Missing logit perplexity confidence": [df["logit_perplexity_confidence"].isna().sum()],
            "Missing verbal linguistic confidence": [df["verbal_linguistic_confidence"].isna().sum()],
            "Accuracy": [(df["extracted_answer"] == df["correct_answer"]).mean()],
            "Accuracy without na": [(acc_no_na["extracted_answer"] == acc_no_na["correct_answer"]).mean()],
            "ECE verbal numerical confidence": [calculate_ece(df["verbal_numerical_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "ECE logit perplexity confidence": [calculate_ece(df["logit_perplexity_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "ECE verbal linguistic confidence": [calculate_ece(df["verbal_linguistic_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "AUROC verbal numerical confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]).values, df["verbal_numerical_confidence"].fillna(0).values),
            "AUROC logit perplexity confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]), df["logit_perplexity_confidence"].fillna(0).values),
            "AUROC verbal linguistic confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]), df["verbal_linguistic_confidence"].fillna(0).values),
            "Mean verbal numerical confidence": [df["verbal_numerical_confidence"].mean()],
            "Mean logit perplexity confidence": [df["logit_perplexity_confidence"].mean()],
            "Mean verbal linguistic confidence": [df["verbal_linguistic_confidence"].mean()],
            "Std verbal numerical confidence": [df["verbal_numerical_confidence"].std()],
            "Std logit perplexity confidence": [df["logit_perplexity_confidence"].std()],
            "Std verbal linguistic confidence": [df["verbal_linguistic_confidence"].std()],
        })
        stats_df = pd.concat([stats_df, new_row], ignore_index=True)
    
display(stats_df.sort_values(by="Model", ignore_index=True))

,Model,Missing answer,Missing verbal numerical confidence,Missing logit perplexity confidence,Missing verbal linguistic confidence,Accuracy,Accuracy without na,ECE verbal numerical confidence,ECE logit perplexity confidence,ECE verbal linguistic confidence,AUROC verbal numerical confidence,AUROC logit perplexity confidence,AUROC verbal linguistic confidence,Mean verbal numerical confidence,Mean logit perplexity confidence,Mean verbal linguistic confidence,Std verbal numerical confidence,Std logit perplexity confidence,Std verbal linguistic confidence
0,Gpt-4.1,30,20,0,0,0.787899,0.789868,0.176778,0.077492,0.0,0.750999,0.678905,0.5,0.965989,0.865391,0.0,0.059353,0.059776,0.0
1,Gpt-4.1-mini,180,171,0,0,0.749418,0.760800,0.168441,0.141744,0.0,0.778065,0.715169,0.5,0.928326,0.891108,0.0,0.062468,0.049969,0.0
2,Gpt-4.1-nano,405,132,0,0,0.613946,0.635332,0.256408,0.235984,0.0,0.721709,0.680920,0.5,0.876526,0.849930,0.0,0.071456,0.058216,0.0
3,Gpt-4o-2024-08-06,54,29,0,0,0.722490,0.725747,0.208816,0.128811,0.0,0.708700,0.639961,0.5,0.932718,0.851301,0.0,0.079690,0.061558,0.0
4,Gpt-4o-2024-08-06_eval_all_3.csv,0,0,0,0,0.666667,0.666667,0.316667,0.271302,0.0,1.000000,1.000000,0.5,0.983333,0.892940,0.0,0.028868,0.038240,0.0
5,Gpt-4o-mini-2024-07-18,31,9,0,0,0.599069,0.600617,0.287581,0.241333,0.0,0.739955,0.674036,0.5,0.886899,0.840402,0.0,0.056000,0.061092,0.0
6,O1-mini-2024-09-12,4,5,12032,0,0.712932,0.713169,0.218326,0.000000,0.0,0.776366,0.500000,0.5,0.931339,NaN,0.0,0.075349,NaN,0.0
7,O3-mini-2025-01-31,421,429,12032,0,0.768201,0.796055,0.160785,0.000000,0.0,0.793498,0.500000,0.5,0.957045,NaN,0.0,0.061770,NaN,0.0
8,O4-mini-2025-04-16,239,237,12032,0,0.815409,0.831934,0.083412,0.000000,0.0,0.799996,0.500000,0.5,0.914043,NaN,0.0,0.090422,NaN,0.0


# Drop NA Across All Models

In [4]:
nan_indices = set()

for file in os.listdir("../results"):
    if file.endswith("csv"):
        df = pd.read_csv("../results/" + file)

        # Find indices where either column has NaN
        indices_with_nan = df[
            df["extracted_answer"].isna() | df["verbal_numerical_confidence"].isna()
        ].index

        # Combine indices across all files
        nan_indices.update(indices_with_nan)

# Convert to sorted list if needed
nan_indices = sorted(nan_indices)
len(nan_indices)

984

In [5]:
stats_df = pd.DataFrame()
for file in os.listdir("../results"):
    if file.endswith("csv"):
        df = pd.read_csv("../results/" + file).drop(index=nan_indices).reset_index(drop=True)
        acc_no_na = df
        new_row = pd.DataFrame({
            "Model": [file.replace("mmlu_pro_", "").replace("_eval_all_None.csv", "").strip().capitalize()],
            "Missing answer": [df["extracted_answer"].isna().sum()],
            "Missing verbal numerical confidence": [df["verbal_numerical_confidence"].isna().sum()],
            "Missing logit perplexity confidence": [df["logit_perplexity_confidence"].isna().sum()],
            "Missing verbal linguistic confidence": [df["verbal_linguistic_confidence"].isna().sum()],
            "Accuracy": [(df["extracted_answer"] == df["correct_answer"]).mean()],
            "Accuracy without na": [(acc_no_na["extracted_answer"] == acc_no_na["correct_answer"]).mean()],
            "ECE verbal numerical confidence": [calculate_ece(df["verbal_numerical_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "ECE logit perplexity confidence": [calculate_ece(df["logit_perplexity_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "ECE verbal linguistic confidence": [calculate_ece(df["verbal_linguistic_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "AUROC verbal numerical confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]).values, df["verbal_numerical_confidence"].fillna(0).values),
            "AUROC logit perplexity confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]), df["logit_perplexity_confidence"].fillna(0).values),
            "AUROC verbal linguistic confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]), df["verbal_linguistic_confidence"].fillna(0).values),
            "Mean verbal numerical confidence": [df["verbal_numerical_confidence"].mean()],
            "Mean logit perplexity confidence": [df["logit_perplexity_confidence"].mean()],
            "Mean verbal linguistic confidence": [df["verbal_linguistic_confidence"].mean()],
            "Std verbal numerical confidence": [df["verbal_numerical_confidence"].std()],
            "Std logit perplexity confidence": [df["logit_perplexity_confidence"].std()],
            "Std verbal linguistic confidence": [df["verbal_linguistic_confidence"].std()],
        })
        stats_df = pd.concat([stats_df, new_row], ignore_index=True)
    
display(stats_df.sort_values(by="Model", ignore_index=True))

KeyError: '[18, 35, 39, 51, 64, 68, 70, 74, 104, 137, 141, 157, 172, 199, 200, 249, 252, 267, 268, 331, 332, 338, 339, 371, 382, 395, 400, 405, 407, 416, 420, 422, 435, 451, 452, 485, 522, 540, 568, 579, 589, 590, 595, 602, 617, 628, 630, 701, 709, 724, 731, 738, 764, 783, 786, 797, 832, 851, 923, 1010, 1209, 1310, 1368, 1432, 1477, 1478, 1484, 1620, 1670, 1686, 1694, 1772, 1806, 1893, 1947, 2019, 2028, 2090, 2711, 2769, 2798, 2825, 2885, 2958, 2966, 3162, 3169, 3210, 3283, 3286, 3336, 3383, 3421, 3435, 3441, 3452, 3459, 3462, 3464, 3468, 3476, 3478, 3490, 3491, 3494, 3495, 3496, 3501, 3522, 3529, 3530, 3531, 3537, 3550, 3552, 3554, 3561, 3564, 3576, 3583, 3587, 3591, 3605, 3614, 3623, 3624, 3636, 3653, 3663, 3666, 3669, 3673, 3682, 3689, 3694, 3715, 3720, 3725, 3736, 3745, 3751, 3754, 3776, 3794, 3796, 3802, 3815, 3825, 3835, 3836, 3846, 3847, 3851, 3853, 3856, 3859, 3868, 3869, 3870, 3875, 3888, 3890, 3900, 3902, 3913, 3930, 3935, 3939, 3941, 3964, 3968, 3972, 3974, 3975, 3977, 3983,